In [47]:
import numpy as np
import operator
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten
#from keras.datasets import imdb
import pandas as pd
import numpy as np
#устанавливаем seed
np.random.seed(42)
import pymorphy2
import re
import sklearn.feature_extraction.text as fex


In [2]:
prodIdColName = "prodId"
subColName = "SubName"
sentColName = "sentence"
trainDF = pd.DataFrame({prodIdColName: [1, 1, 1,2], subColName: ['телефон', 'смартвон', 'жизнь', 'жизнь'], sentColName:['SENTтелефон', 'SENTсмартвон', 'SENT00жизнь', 'SENT2жизнь'] })

In [3]:
testDF = pd.DataFrame({prodIdColName: [3, 3, 3,4], subColName: ['телефон', 'смартвон', 'жизнь', 'жизнь'], sentColName:['3SENTтелефон', '3SENTсмартвон', '3SENT00жизнь', '3SENT2жизнь'] })

In [4]:
full = trainDF.append(testDF)

In [5]:
df_train = pd.read_csv('dataset1.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)


In [6]:
textColName = "TEXT"
negatColName =  "DRAWBACKS"
positColName = "BENEFITS"
ratingColName = "RATING"
catColName = "CATEGORY_ID"
brandColName = "BRAND_ID"
catNameColName= "CATEGORY_NAME"
tovarIdColName = "PRODUCT"
df_train.head()

,Unnamed: 0,PRODUCT,CATEGORY_ID,BRAND_ID,CATEGORY_NAME,BRAND_NAME,TYPE,SUBJECT,TEXT,POSTED_DATE,RATING,BENEFITS,DRAWBACKS,RECOMMENDED,LIKES_COUNT,DISLIKES_COUNT
0,0,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Что особенно ценно - при профилактике принтера...,23.10.16 22:26:00,5.0,"хорошая скорость печати, двусторонняя печать, ...","Рядом ""на стол"" не поставишь, место много зани...",1.0,2.0,0.0
1,1,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Он у нас три месяца в офисе стоит уже - большо...,17.04.16 11:17:42,5.0,NaN,NaN,NaN,0.0,2.0
2,2,30024725,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,"Просто небо и земля с тем принтером, который у...",15.04.16 20:22:47,4.0,NaN,NaN,NaN,0.0,1.0
3,3,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,С такой вроде бы навороченной техникой все смо...,У нас в кабинете недавно его поставили. Первое...,18.04.16 20:01:57,5.0,NaN,NaN,NaN,0.0,0.0
4,4,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,NaN,Очень многофункциональная штука. Есть все нео...,23.04.16 12:03:39,5.0,NaN,NaN,NaN,1.0,0.0


In [7]:
morph = pymorphy2.MorphAnalyzer()

def normalize(text):
    return ' '.join([morph.parse(str(w))[0].normal_form for w in text.split()])
def clean_text(text):
    text = str(text)
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-\'>><`', ' ', text) #deleting symbols  
    text = re.sub('-', ' ', text) #deleting symbols  
    #text = ' '.join(word[:] for word in text.split() if len(word)>3)
    #text = text.encode("utf-8")
    return text 

In [8]:
#df_train[textColName]=df_train[textColName].apply(clean_text).apply(normalize)

In [12]:
#import tparser
from tparser import tomita
from tparser.tomita_parser import text_parse
import pandas as pd

In [14]:
catidNameForBuild = "SMARTPHONES"
category_data = df_train[df_train[catNameColName].str.contains(catidNameForBuild)==True]

In [ ]:
textAndProd = category_data.ix[:,[textColName,tovarIdColName]]
textAndProdGrS = textAndProd.groupby(tovarIdColName )[tovarIdColName, textColName].agg(lambda x: "{%s}" % ', '.join(x)).reset_index()
print(len(textAndProd))
print(len(textAndProdGrS))

In [139]:
mask = df_train[df_train[tovarIdColName] == 30025022 ]

In [ ]:
textAndProdGrSnORM = textAndProdGrS[textColName].apply(clean_text).apply(normalize)

In [128]:
textAndProdGrS[textColName]

0       {Прикольная штука, давно хотела что-либо подоб...
1       {Классный сотовый правда нет секундомера)А так...
2       {отличный коммуникатор.\nпочти год прошел,как ...
3       {Солидный смарт. Понимаю, 2,5 разъем не для му...
4       {хороший аппарат, согласен! после него как-то ...
5       {Купил этот коммуникатор в сентябре 2009. Хочу...
6       {В двух словах хорошая моделька. Странно,но ра...
7       {Этот аппарат без сомнения хорош) У знакомого ...
8       {найдите подобный аппарат в пределах 10 тыс)))...
9       {Телефон просто замечательный. Внешность, функ...
10      {Подарили такой коммуникатор на 18 лет. Вообще...
11      {никаких претензий!!!, Очень хороший телефон о...
12      {клевый смарт, Пользуюсь этим телефоном 6 меся...
13      {Имейте ввиду, что пленка поляризационная, инф...
14      {Телефон такого типа у меня первый, поэтому ск...
15      {Телефон сам по себе нормальный, но через год,...
16      {Отличный мобайл, жаль что экран маловат и g-s...
17      {Я взя

In [129]:
tomitaCol = textAndProdGrS[1000:1005][textColName].apply(text_parse)

In [137]:
textAndProdGrS[1000:1002]

,PRODUCT,TEXT
1000,30025022,"{Очень порадовал аккумулятор, хватает более че..."
1001,30025023,"{Отличный дизайн, яркий экран, камера четкая,з..."


In [131]:
ind = 1003
#print(textAndProdGrS[textColName].iloc[ind])
tomitaCol[ind]

array([['АЙФОН', 'ПОСЛЕДНИЙ ЛЕТ 5 ПОЛЬЗОВАЛАСЬ ИСКЛЮЧИТЕЛЬНО АЙФОН'],
       ['ПОЛОМКА', 'ПОСЛЕ ПОЛОМКА'],
       ['СТОИМОСТЬ',
        '5ЫЙ ШЕСТОЙ НЕ ХОТЕЛОСЬ ПОКУПАТЬ ИЗ-ЗА ЕГО НЕРЕАЛЬНО ЗАВЫШЕННОЙ СТОИМОСТЬ'],
       ['САМСУНГ', 'МОЕ ПРЕДСТАВЛЕНИЕ ОБ АНДРОИДЕ БЫЛО ОМРАЧНЕНО САМСУНГ'],
       ['ВЗГЛЯД', 'ВЗГЛЯД НА КОТОРЫЕ НАЧИНАЛИ БОЛЕТЬ ГЛАЗА ОТ СЛИШКОМ'],
       ['ЦВЕТ', 'ЯРКИЙ ЦВЕТ'],
       ['АНДРОИД', 'АНДРОИД ТАКОЙ БЕЗВКУСНЫЕ'],
       ['ТЕЛЕФОН',
        'ТЕЛЕФОН Я СРАЗУ ВЛЮБИЛАСЬ В БЕЛУЮ ПОВЕРХНОСТЬ ИЗ ОООЧЕНЬ ПРИЯТНОГО'],
       ['МАТЕРИАЛ', 'МАТОВЫЙ МАТЕРИАЛ'],
       ['ЭКРАН', 'ЭКРАН И ЦВЕТА ЕСТЕСТВЕННЫЕ , КАЧЕСТВЕННЫЕ'],
       ['ВЗГЛЯД', 'НЕ ОТТОРГАЮЩИЕ ВЗГЛЯД'],
       ['ОЩУЩЕНИЕ', 'ОЩУЩЕНИЕ ДАЖЕ ЛУЧШЕ'],
       ['АЙФОН', '5ЫЙ АЙФОН'],
       ['ПРИЛОЖЕНИЕ', 'ПРИЛОЖЕНИЕ РАБОТАТЬ БЫСТРО'],
       ['РАЗМЕР', 'РАЗМЕР ТЕЛЕФОН ИДЕАЛЬНЫЙ'],
       ['ПОЧТА', 'ОЧЕНЬ УДОБНО ЛАЗИТЬ В ИНТЕРНЕТЕ ИЛИ ПОЛЬЗОВАТЬСЯ ПОЧТА'],
       ['АЙФОН', 'АЙФОН 5 КАЖЕТСЯ ТЕПЕРЬ МАЛЕНЬКИМ И НЕУДОБНЫМ

In [132]:
tfidf = fex.TfidfVectorizer()

In [133]:
tfIdfMatrix = tfidf.fit_transform(textAndProdGrSnORM)
ftNames = tfidf.get_feature_names()

In [136]:
densM = tfIdfMatrix.todense()
densM.shape

(2166, 23529)

## texcC = 2000
valD = dict()
for i in range(ind,ind+1):
    col = densM[i]
    col = col.tolist()
    col = col[0]
    #for i,v in enumerate(col):
    #   print(str(i) + " v "+ str(v))
    valD[ind] = { ftNames[i]:v for i,v in enumerate(col) if v>0  }
#print (valD)
sorted_topWords = sorted(valD[ind].items(), key=operator.itemgetter(1), reverse=True)
for i in range(ind,ind+1):
    sorted_topWords = sorted(valD[i].items(), key=operator.itemgetter(1), reverse=True)
    for w in sorted_topWords:
        if w[0].upper() in slAll:
            was = False
            for tw in tomitaCol[ind]:
                if tw[0] == w[0].upper():
                    was=True
                    print(tw[1])
            if was:
                print(w)


In [120]:
sublist = [x[::, 0] for x in tomitaCol.values]

In [84]:
slAll=list()
for sl in sublist:
    type(sl)
    slAll.append(list(sl))

['АККУМУЛЯТОР',
 'ИСПОЛЬЗОВАНИЕ',
 'ОБОЛОЧКА',
 'ИНТЕРНЕТ',
 'ЛАГ',
 'АППАРАТ',
 'КАРТА',
 'АКСЕССУАР',
 'ДЕНЬГА',
 'БРАК',
 'МАГАЗИН',
 'КОМПАНИЯ',
 'ЧЛЕН СЕМЬИ',
 'ПРЕИМУЩЕСТВО',
 'МАМ',
 'СМАРТФОН-ФЛАГМАН',
 'ВЧЕРА',
 'Я',
 'МОДЕЛЬ',
 'ДО',
 'ЦЕНА',
 'ГОД',
 'АВТОНОМНОСТЬ',
 'РАЗ',
 'РАЗ',
 'ПРЕИМУЩЕСТВО',
 'СЕТЬ',
 'ИНТЕРНЕТ',
 'В',
 'И',
 'ТУТ',
 'МАКСИМУМ',
 'ПРОЦЕССОР',
 'ПОКРЫТИЕ',
 'АППАРАТ',
 'СМАРТФОН',
 'БАТАРЕЯ',
 'БАТАРЕЯ',
 'ИСПОЛЬЗОВАНИЕ',
 'ДЕНЬ',
 'СМАРТФОН',
 'ГИГ',
 'КАМЕРА',
 'И',
 'В',
 'МАРКА',
 'СЛУХ',
 'СМАРТФОН',
 'ПОКУПКА',
 'ПОКУПКА ПЛАНШЕТА',
 'НЕТ',
 'ПРИОБРЕТЕНИЕ',
 'К',
 'БЫЛЬ',
 'ЭТА МОДЕЛИ',
 'УЖ',
 'ДРУГОЙ АППАРАТ',
 'ДЕНЬГА',
 'ПРОЦЕССОР',
 'АППАРАТ',
 'АППАРАТ',
 'ДЕНЬГА',
 'СЫН',
 'ТЕЛЕФОН ОТЦУ',
 'ЭКРАН',
 'БАТАРЕЙКА',
 'ИСПОЛЬЗОВАНИЕ',
 'ИГРА',
 'ПОЛЬЗОВАНИЕ',
 'ТЕЛЕФОН',
 'СМАРТФОН',
 'ВИД',
 'СЫН',
 'РАЗ',
 'О',
 'ПОДЗАРЯДКА',
 'ИНТЕРНЕТ',
 'КАМЕРА',
 'В',
 'ЭТА МОДЕЛЬ',
 'СТЕКЛО',
 'ДЕНЬГА',
 'АККУМУЛЯТОР',
 'ОШИБКА',
 'КОМПЛЕКТАЦИЯ',
 'ФУНКЦИ

In [113]:
sublistInd = [i for i,n in enumerate(ftNames)  if n.upper() in slAll  ]

In [91]:
sublistInd

[3683,
 3954,
 4010,
 4452,
 4884,
 4891,
 5330,
 5378,
 5387,
 5705,
 5823,
 6006,
 6950,
 7291,
 7913,
 8243,
 8391,
 8605,
 8657,
 9958,
 10258,
 10616,
 10835,
 11250,
 11411,
 11585,
 11630,
 11787,
 11885,
 12616,
 14907,
 15471,
 15480,
 15862,
 15933,
 16476,
 16653,
 16668,
 16870,
 17105,
 17304,
 17422,
 17423,
 17431,
 17608,
 18209,
 18356,
 18380,
 18385,
 18844,
 19521,
 19607,
 20464,
 20578,
 20690,
 20737,
 21247,
 21258,
 21566,
 21800,
 21949,
 23613,
 24004,
 24097,
 25018,
 26170,
 26368,
 28180,
 29240,
 29409,
 29471,
 29533,
 29874,
 29944,
 30786,
 30903,
 31016,
 31652,
 31956,
 32585,
 32633,
 33037,
 34007,
 34046,
 34901,
 34966,
 35411,
 35737,
 35917,
 36259,
 36466,
 37229,
 37517,
 38049,
 38228,
 39338,
 39794,
 39923,
 40328,
 40638,
 41063,
 41328,
 41563,
 41709,
 41987,
 42521,
 42555,
 42631,
 42700,
 44628,
 44762,
 45217,
 45352,
 45456,
 45523,
 45924,
 45959,
 46236,
 46409,
 46527,
 47062,
 47065,
 47777,
 47825,
 48373,
 48474,
 49228,
 492